In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/

In [ ]:
token = ''

In [ ]:
!git clone https://{token}@github.com/kytomic/fake-image-classification.git

In [ ]:
%cd /content/drive/MyDrive/fake-image-classification/

In [ ]:
!git status

In [ ]:
!git checkout -b build_cnn

In [ ]:
!git status

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [ ]:
!pwd

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "real_and_fake_face"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

In [ ]:
train_dir = image_path / "training"
test_dir = image_path / "test"

train_dir, test_dir

In [ ]:
import random
from PIL import Image

random.seed(42)
image_path_list = list(image_path.glob("*/*/*.jpg"))
random_image_path = random.choice(image_path_list)
image_class = random_image_path.parent.stem
img = Image.open(random_image_path)


print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

In [ ]:
train_dir

In [ ]:
train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

In [ ]:
ls -a data/real_and_fake_face/training

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=1,
                              num_workers=1,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=1,
                             num_workers=1,
                             shuffle=False)

In [ ]:
class FakeImageModel(nn.Module):
  def __init__(self, input_shape, hidden_units, output_shape):
    super().__init__()
    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*16*16, out_features=output_shape)
    )

  def forward(self, x):
    x = self.block_1(x)
    # print('Block 1: ', x.shape)
    x = self.block_2(x)
    # print('Block 2: ', x.shape)
    x = self.classifier(x)
    # print('Classifier: ', x.shape)
    return x

torch.manual_seed(42)
model = FakeImageModel(input_shape=3, hidden_units=10, output_shape=len(class_names))
model

In [ ]:
def accuracy(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100

    return acc

In [ ]:
from tqdm.auto import tqdm
torch.manual_seed(42)
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.1)

for epoch in tqdm(range(epochs)):
  print(f'Epoch: {epoch} -----')
  train_loss = 0

  for batch, (X, y) in enumerate(train_dataloader):
    model.train()
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 400 == 0:
      print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  train_loss /= len(train_dataloader)
  print(f"Train loss: {train_loss:.5f}")

  model.eval()
  with torch.inference_mode():
    test_loss, test_acc = 0.0, 0.00

    for X, y in test_dataloader:
      test_pred = model(X)
      test_loss += loss_fn(test_pred, y)
      test_acc += accuracy(y_true=y, y_pred=test_pred.argmax(dim=1))

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
model.eval()
with torch.inference_mode():
  test_loss, test_acc = 0.0, 0.0
  for X, y in test_dataloader:
    test_pred = model(X)
    test_loss += loss_fn(test_pred, y)
    test_acc += accuracy(y_true=y, y_pred=test_pred.argmax(dim=1))

  test_loss /= len(test_dataloader)
  test_acc /= len(test_dataloader)
  print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
!git status

In [ ]:
!git add main.ipynb

In [ ]:
!git commit -m 'added model and dataset'

In [ ]:
!git push --set-upstream origin build_cnn

In [ ]:
!git reset --hard HEAD~1